In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("kmeans_moments_lattice.jl")
using .LatticeGeneration
using JLD2
using SDDP, HiGHS, Plots, XLSX, Mustache, PrettyTables

In [24]:
includet("goal_investing_sddp_model.jl")
using .GoalInvestingSDDP

In [39]:
scenario_grid = load_object("scenario_grid.jld2")
println(size(scenario_grid.probs))
println(size(scenario_grid.states))

(6,)
(5, 9, 3)


In [40]:
graph = SDDP.MarkovianGraph(scenario_grid.probs[1:end-1])

Root
 (0, 1)
Nodes
 (1, 1)
 (2, 1)
 (2, 2)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (2, 7)
 (2, 8)
 (2, 9)
 (3, 1)
 (3, 2)
 (3, 3)
 (3, 4)
 (3, 5)
 (3, 6)
 (3, 7)
 (3, 8)
 (3, 9)
 (4, 1)
 (4, 2)
 (4, 3)
 (4, 4)
 (4, 5)
 (4, 6)
 (4, 7)
 (4, 8)
 (4, 9)
 (5, 1)
 (5, 2)
 (5, 3)
 (5, 4)
 (5, 5)
 (5, 6)
 (5, 7)
 (5, 8)
 (5, 9)
Arcs
 (0, 1) => (1, 1) w.p. 1.0
 (1, 1) => (2, 1) w.p. 0.157
 (1, 1) => (2, 2) w.p. 0.10318
 (1, 1) => (2, 3) w.p. 0.0978
 (1, 1) => (2, 4) w.p. 0.07771
 (1, 1) => (2, 5) w.p. 0.13828
 (1, 1) => (2, 6) w.p. 0.10484
 (1, 1) => (2, 7) w.p. 0.12405
 (1, 1) => (2, 8) w.p. 0.07065
 (1, 1) => (2, 9) w.p. 0.12649
 (2, 1) => (3, 1) w.p. 0.09159235668789809
 (2, 1) => (3, 2) w.p. 0.08452229299363058
 (2, 1) => (3, 3) w.p. 0.1346496815286624
 (2, 1) => (3, 4) w.p. 0.13522292993630572
 (2, 1) => (3, 5) w.p. 0.1927388535031847
 (2, 1) => (3, 6) w.p. 0.09789808917197453
 (2, 1) => (3, 7) w.p. 0.07738853503184713
 (2, 1) => (3, 8) w.p. 0.15369426751592358
 (2, 1) => (3, 9) w.p. 0.0322929936

In [43]:
data_source = DataFrame(XLSX.readtable("./goal_data.xlsx", "goal_data_single_g_3y")) 
data = round.(data_source[1:6,:],digits=3)

Row,t,minimum_limit,acceptable_limit,desired_limit,inflows,priority,minimum_utility,acceptable_utility,desired_utility,above_desired_utility,cashflows
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,15.0,20.0,23.0,16.0,0.2,0.333,0.2,0.067,0.003,-42.0
2,2.0,18.0,24.0,27.0,34.0,0.2,0.278,0.167,0.067,0.003,-7.0
3,3.0,18.0,24.0,27.0,33.0,0.2,0.278,0.167,0.067,0.003,-6.0
4,4.0,18.0,24.0,27.0,33.0,0.2,0.278,0.167,0.067,0.003,-6.0
5,5.0,18.0,24.0,27.0,33.0,0.2,0.278,0.167,0.067,0.003,-6.0
6,6.0,84.0,132.0,163.0,16.0,1.0,0.298,0.104,0.032,0.003,147.0


In [44]:
model_data = GoalsData3(
    data.minimum_limit,
    data.acceptable_limit,
    data.desired_limit,
    data.inflows,
    data.minimum_utility,
    data.acceptable_utility,
    data.desired_utility,
    data.above_desired_utility,
    35.0,
    0.0029
    )

GoalsData3([15.0, 18.0, 18.0, 18.0, 18.0, 84.0], [20.0, 24.0, 24.0, 24.0, 24.0, 132.0], [23.0, 27.0, 27.0, 27.0, 27.0, 163.0], [16.0, 34.0, 33.0, 33.0, 33.0, 16.0], Float32[0.333, 0.278, 0.278, 0.278, 0.278, 0.298], Float32[0.2, 0.167, 0.167, 0.167, 0.167, 0.104], Float32[0.067, 0.067, 0.067, 0.067, 0.067, 0.032], Float32[0.003, 0.003, 0.003, 0.003, 0.003, 0.003], 35.0, 0.0029f0)

In [45]:
model = asset_management_alm(model_data, scenario_grid)

A policy graph with 46 nodes.
 Node indices: (1, 1), ..., (6, 9)


In [46]:
SDDP.train(model; iteration_limit = 400, log_frequency = 50)


-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 46
  state variables : 3
  scenarios       : 5.90490e+04
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [18, 18]
  AffExpr in MOI.EqualTo{Float64}         : [5, 5]
  VariableRef in MOI.GreaterThan{Float64} : [15, 15]
  VariableRef in MOI.LessThan{Float64}    : [3, 4]
numerical stability report
  matrix range     [9e-01, 2e+00]
  objective range  [3e-03, 1e+00]
  bounds range     [3e+00, 8e+01]
  rhs range        [2e+01, 4e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
------------------------------------------

In [48]:
result = simulate_policy(model)

Main.GoalInvestingSDDP.Optimistion_Result(-57.9266, -58.0219, 0.1009, [18.535342662504497 24.0 … 24.0 136.31940488697452; 18.535342662504497 22.861102637467667 … 24.0 90.27886901256821; … ; 18.535342662504497 22.861102637467667 … 24.0 92.13339448759751; 18.535342662504497 24.0 … 20.96750486675486 114.0987777994722], [16.417047826777868 16.417047826777868 … 16.417047826777868 16.417047826777868; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 38.05215338733643 28.92564215358688 … 28.92564215358688 31.7313784726017; 0.0 0.0 … 0.0 1.2391938932459097; 0.0 0.0 … 0.0 0.0;;; 51.97815149469203 44.636780103048814 … 38.15704875686087 40.48227241315219; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 57.01598862897328 59.75107777675287 … 54.87896404145404 51.23537590677638; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 84.80760586433976 68.61938054955343 … 81.96527449919242 65.21914879004825; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [49]:
# 3y returns
result.lower_bound

-57.9266

In [34]:
result.lower_bound

-60.1278

In [50]:

function calculate_perc(data, perc = [0.05, 0.25, 0.5, 0.75, 0.95])
    n_scenarios, n_stages = size(data)
    
    result = zeros(n_stages, length(perc)) 

    for t in 1:n_stages
       result[t,:] = quantile(data[:,t], perc)
    end

    return DataFrame(round.(result,digits=2),string.(perc))
  
end

calculate_perc (generic function with 2 methods)

In [38]:
calculate_perc(result.consumption_dist)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,8.0,8.0,8.0,8.0,8.0
2,8.0,8.0,8.0,8.0,8.0
3,8.0,8.0,8.0,8.0,9.0
4,8.0,8.0,8.0,8.45,9.0
5,8.0,8.0,8.0,9.0,9.0
6,8.0,8.0,8.0,9.0,9.0
7,8.0,8.0,8.0,9.0,9.0
8,8.0,8.0,8.52,9.0,9.0
9,8.0,8.0,9.0,9.0,9.0


In [51]:
# ret 3y

calculate_perc(result.consumption_dist)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,18.54,18.54,18.54,18.54,18.54
2,22.86,24.0,24.0,24.0,24.0
3,22.59,24.0,24.0,24.0,24.0
4,20.25,24.0,24.0,24.0,27.0
5,18.0,24.0,24.0,24.0,27.0
6,83.99,102.66,118.75,139.06,172.66


In [56]:
# ret 3y
calculate_perc(result.assets_dist[2,:,:])


Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,27.32
3,0.0,0.0,0.0,0.0,3.37
4,0.0,0.0,0.0,0.0,1.41
5,0.0,0.0,0.0,0.0,46.9
6,0.0,0.0,0.0,0.0,0.0


In [35]:
report = print_report(result, model_data, "c:\\Users\\matsz\\programowanie\\Optymalizacja_portfela\\julia_msp\\reports\\tpl.html")

MethodError: MethodError: no method matching render(::String, ::Main.GoalInvestingSDDP.Optimistion_Result; TITLE::String, input_data::GoalsData3)

Closest candidates are:
  render(::AbstractString, ::Any; tags) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:55
  render(!Matched::Mustache.MustacheTokens, ::Any) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:30
  render(::AbstractString; kwargs...)
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:56


- simulate policy
- generate report summary
- save report